# ML Pipeline Preparation

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [66]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
import sklearn
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
from sqlalchemy import create_engine
engine = create_engine('sqlite:///MyDisasterMessage.db')
df = pd.read_sql_table('MyDisasterMessage', engine)
df.sample(10)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
23521,27229,This project began in October 2008 with cash s...,None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19995,23297,- Maiduguri hit - With Boko Haram squeezed out...,None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24449,28269,The medical help involves the setting up of U....,None,news,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4600,5236,We are living in Carrefour Truitier. We reprse...,"we leaving in carrefour truitier, we represent...",direct,1,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
7129,8054,I need to know when DGI will start again with ...,M bezwen konnen kil DGI ki ouv yo ap koumanse ...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1165,1392,our hous is destroyed we have wounded and preg...,KAY NOU KRAZE NOU GEN MOUN BLESE FANM ANSENT N...,direct,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1
22238,25776,The partnership is supporting the rollout of t...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24594,28435,"Following a period of decline, the number of r...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25494,29432,Organized voluntary repatriation and assisted ...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2277,2625,i would like to know when the american embassy...,Mwen ta renmen konnen kil anbasad ameriken ap ...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# define X and y
X = df['message']
y = df.iloc[:, 4:]
print(X.head())
y.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process  text data

In [4]:

def tokenize(text):
    '''
    Tokenize text, which composed of:
    1. replace url to word 'urlplaceholder'
    2. word_tokenize text
    3. relace words to lower case
    4. lemmatizer words
    return list of cleaned tokens
    '''
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    detected_urls = re.findall(url_regex, text)
    
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) is used for predicting multiple target variables.

In [5]:
# Build training pipeline 
def build_model():
    pipeline = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(RandomForestClassifier(),n_jobs=-1))
        ])


    #cv = GridSearchCV(pipeline, param_grid=parameters)
    #cv = RandomizedSearchCV(pipeline, param_distributions=parameters )
    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = build_model()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. Do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [30]:
# check global accuracy first 
print('Overal accuracy is:', '{:.2f}'.format((y_test==y_pred).sum().sum()/(y_pred.shape[0]*y_pred.shape[1])*100), '%')

Overal accuracy is: 94.22 %


In [12]:
print(classification_report(y_test.iloc[:, 0], y_pred[:, 0]))

             precision    recall  f1-score   support

          0       0.63      0.34      0.44      1547
          1       0.82      0.94      0.87      4968
          2       1.00      0.13      0.23        39

avg / total       0.77      0.79      0.77      6554



In [54]:
# first column y_test has values of 0, 1, 2, the rest are all binary outputs
for idx, col in enumerate(y_test.columns.values):
    print('--------------------------------')
    print('Feature --', col, ':\n')
    print(classification_report(y_test[col], y_pred[:, idx]))

--------------------------------
Feature -- related :

             precision    recall  f1-score   support

          0       0.63      0.34      0.44      1547
          1       0.82      0.94      0.87      4968
          2       1.00      0.13      0.23        39

avg / total       0.77      0.79      0.77      6554

--------------------------------
Feature -- request :

             precision    recall  f1-score   support

          0       0.89      0.99      0.93      5407
          1       0.87      0.40      0.55      1147

avg / total       0.88      0.88      0.87      6554

--------------------------------
Feature -- offer :

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

--------------------------------
Feature -- aid_related :

             precision    recall  f1-score   support

          0       0.72    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [53]:
 # first column y_test has values of 0, 1, 2, the rest are all binary outputs, remove first column to display together
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), 
                            target_names=y_test.columns.values.tolist()[1:]))

                        precision    recall  f1-score   support

               request       0.87      0.40      0.55      1147
                 offer       0.00      0.00      0.00        26
           aid_related       0.76      0.51      0.61      2717
          medical_help       0.66      0.07      0.13       543
      medical_products       0.69      0.10      0.17       342
     search_and_rescue       0.25      0.01      0.01       166
              security       0.57      0.04      0.07       112
              military       0.69      0.10      0.18       214
           child_alone       0.00      0.00      0.00         0
                 water       0.96      0.23      0.37       392
                  food       0.85      0.29      0.43       720
               shelter       0.85      0.28      0.42       593
              clothing       0.91      0.20      0.33       100
                 money       0.57      0.02      0.04       172
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [55]:
model.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [65]:
parameters = {
        'clf__estimator__max_depth':[10, 50],
        'clf__estimator__min_samples_split': [2, 4],
        'clf__estimator__n_estimators': [50, 100],
        #'clf__estimator__min_samples_leaf':[2, 5, 10]
        }

model_opt = build_model()
cv = GridSearchCV(model_opt, param_grid=parameters)
cv.fit(X_train, y_train)


In [61]:
y_pred_opt = cv.predict(X_test)

In [62]:
print("\nBest Parameters:", cv.best_params_)


Best Parameters: {'clf__estimator__max_depth': 50, 'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 100}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  


In [63]:
# check global accuracy first 
print('Overal accuracy is:', '{:.2f}'.format((y_test==y_pred_opt).sum().sum()/(y_pred.shape[0]*y_pred.shape[1])*100), '%')

Overal accuracy is: 94.81 %


In [64]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred_opt]), 
                            target_names=y_test.columns.values.tolist()[1:]))

                        precision    recall  f1-score   support

               request       1.00      0.44      0.61      1060
                 offer       1.00      0.08      0.15        36
           aid_related       0.95      0.67      0.79      2689
          medical_help       0.98      0.10      0.19       563
      medical_products       1.00      0.15      0.26       341
     search_and_rescue       1.00      0.02      0.03       170
              security       0.00      0.00      0.00        86
              military       1.00      0.14      0.25       221
           child_alone       0.00      0.00      0.00         0
                 water       1.00      0.20      0.34       425
                  food       1.00      0.33      0.50       709
               shelter       1.00      0.16      0.28       584
              clothing       1.00      0.22      0.36       101
                 money       1.00      0.05      0.10       154
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Although the overall accuracy is not improved dramatically, however the average precision improves from 0.71 to 0.98, recall improves from 0.28 to 0.31, which is remarkable imporvement. 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [67]:
# add additional feature: starting verb or not. 
# use gradiant boost classifier instead
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [69]:

pipeline_new = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1))
    ])

model_new = pipeline_new
model_new.fit(X_train, y_train)
y_pred = model_new.predict(X_test)

In [70]:
# check global accuracy first 
print('Overal accuracy is:', '{:.2f}'.format((y_test==y_pred).sum().sum()/(y_pred.shape[0]*y_pred.shape[1])*100), '%')

Overal accuracy is: 94.10 %


In [72]:
print(classification_report(y_test.iloc[:,1:].values, np.array([x[1:] for x in y_pred]), 
                            target_names=y_test.columns.values.tolist()[1:]))

                        precision    recall  f1-score   support

               request       0.80      0.38      0.52      1060
                 offer       0.00      0.00      0.00        36
           aid_related       0.75      0.52      0.61      2689
          medical_help       0.60      0.07      0.12       563
      medical_products       0.72      0.08      0.14       341
     search_and_rescue       0.30      0.02      0.03       170
              security       0.50      0.01      0.02        86
              military       0.62      0.07      0.13       221
           child_alone       0.00      0.00      0.00         0
                 water       0.84      0.20      0.33       425
                  food       0.87      0.36      0.51       709
               shelter       0.77      0.14      0.24       584
              clothing       0.70      0.07      0.13       101
                 money       0.64      0.05      0.08       154
        missing_people       0.50      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [73]:
import pickle


In [74]:
pickle.dump(cv, open('model.p', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.